# 2019-01-31 - Generer une tuning curve avec les MC
Je sais pas si c'est vraiment utile, mais mon intuition est qu'on va devoir savoir dans quoi on a planté l'electrode pour pouvoir faire des analyses avec le modèle. Donc pour connaitre la préférence à l'orientation de la colonne corticale, on va faire un ensemble d'orientations aléatoires avec le même bandwith bien discriminable et regarder la réponse max.

# B_theta n'a pas d'effet, il faut le spécifier dans la fonction
# Les frames sont toutes fixés à la durée moyenne, pas à la durée aléatoire

In [1]:
import os
import numpy as np
import MotionClouds as mc
import datetime
import matplotlib.pyplot as plt
import csv

downscale = 1
fx, fy, ft = mc.get_grids(mc.N_X//downscale, mc.N_Y//downscale, mc.N_frame//downscale)

now = datetime.datetime.now()
strtime = now.strftime("%Y-%m-%d_%H:%M:%S")   

mc.figpath = os.path.join('sequences/Tuning_curve%s'% strtime)
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)

In [2]:
def make_one_block(N_X, N_Y, seed, B_thetas, N_frame_total,  
                   N_frame_mean=6, N_theta=12, contrast=1.,
                  refresh_rate = 120, duration = 60,mean_duration = 3):
    fx, fy, ft = mc.get_grids(N_X, N_Y, 1)

    rng = np.random.RandomState(seed)

    N_frame = 0
    im = np.zeros((N_X, N_Y, 0))
    disk = mc.frequency_radius(fx, fy, ft) < .5
    itr = 0
    
    if duration != None and mean_duration != None and refresh_rate != None :
        N_frame_total = refresh_rate * duration
        N_frame_mean = refresh_rate * mean_duration
    
    while N_frame < N_frame_total:
        itr +=1
        N_frame_sub = N_frame_mean
        theta = np.int(rng.rand()*N_theta) * np.pi / N_theta
        B_theta = np.pi/32
        mc_i = mc.envelope_gabor(fx, fy, ft, 
                                         V_X=0., V_Y=0., B_V=0., 
                                         sf_0=sf_0, B_sf=B_sf, 
                                         theta=theta, B_theta=B_theta)
        im_ = np.zeros((N_X, N_Y, 1))
        im_ += mc.rectif(mc.random_cloud(mc_i, seed=seed+N_frame), contrast=contrast)
        im_ *= disk # masking outside the disk 
        im_ += .5*(1-disk) # gray outside the disk
        im_ = im_ * np.ones((1, 1, N_frame_sub)) #  expand to N_frame_sub frames

        im_[6:20,6:20,:] = 0
        im_[6:20,6:20,-1] = 1
        im = np.concatenate((im, im_), axis=-1) # montage
        N_frame = im.shape[-1]

        #data appending
        with open(mc.figpath+'/'+'sequence.csv', "a+") as writer_file:
            data_writer = csv.writer(writer_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            data_writer.writerow([itr, N_frame, N_frame/refresh_rate, 
                                  theta, B_theta, sf_0, B_sf, N_X, N_Y, N_frame_total, refresh_rate])

    return im[:, :, :N_frame_total], itr

#Parameters
N_X = fx.shape[0]
width = 29.7*256/1050 
N_X, N_Y = mc.N_X, mc.N_Y

N_frame_total = 200
N_frame_mean = 12
N_theta = 12

sf_0 = 4.*width/N_X
B_V = .0     # BW temporal frequency (speed plane thickness)
B_sf = sf_0   # BW spatial frequency

B_thetas = [np.pi/32, np.pi/16, np.pi/8, np.pi/4] #bandwidth opening choices
seed = 42 #standardized random vector

#Overrides N_frame total, nframe mean
refresh_rate = 24 #Hz
duration = 60 #s
mean_duration = 2 #s

'''refresh_rate = None #Hz
duration = None #s
mean_duration = None #s'''

vext = '.mp4'

#refresh rate and duration and mean duration override the N_frame total;N_frame_mean unless left to None
im = make_one_block(N_X, N_Y, seed=seed, B_thetas=B_thetas,
                    N_frame_total=N_frame_total, N_frame_mean=12, N_theta=12,
                   refresh_rate = refresh_rate, duration = duration, mean_duration = mean_duration)

mc.anim_save(im[0], os.path.join(mc.figpath, 'MC_files'), figpath=mc.figpath, vext = vext, T_movie = duration)

# Protocol description

In [3]:
print('Generation of a tuning curve using motionclouds as gratings')
print('Total duration of %s frames, which is %.3fs at %s Hz' % (N_frame_total, N_frame_total/refresh_rate, refresh_rate))
print('A total of %s blocks where generated, with the mean duration of each block being %s frames' % (im[1],N_frame_mean))
print('%s differents angles where shown, with %s possible bandwiths' % (N_theta, len(B_thetas)))
print('-----------------\nAll files were saved in /%s with format %s' % (mc.figpath, vext))

Generation of a tuning curve using motionclouds as gratings
Total duration of 200 frames, which is 8.333s at 24 Hz
A total of 30 blocks where generated, with the mean duration of each block being 12 frames
12 differents angles where shown, with 4 possible bandwiths
-----------------
All files were saved in /sequences/Tuning_curve2019-01-31_10:07:58 with format .mp4
